In [3]:
from openai import OpenAI

# Setup
api_key = ""
base_url = "https://gpt.uni-muenster.de/v1"
model = "mistral-small"  # model used

client = OpenAI(api_key=api_key, base_url=base_url)


# Define scenario
scenario = """
A customer places an order on an e-commerce website. 
The system checks inventory. 
If the item is in stock, customer is informed to pay for the product.
Customer pays for the order.
The payment gateway confirms the payment transaction. 
Finally, the system sends order confirmation email to the customer.
"""

# Create the prompt
prompt = f"""
You are an expert in Subject-Oriented Business Process Modeling using the Parallel Activity Specification Schema (PASS).

Given the following scenario:

\"\"\"{scenario}\"\"\"

Your task is to generate a PASS model in two parts:

### 1. Subject Interaction Diagram (SID):
- Identify all the **subjects** involved.
- Describe their **interactions** using messages in **noun form only** (e.g., "Order", "Inventory Check Request", "Confirmation Email").
- Avoid using verbs like “place”, “send”, “check”, or “confirm”.
- Use noun phrases that represent the content of the interaction, not the action.
- Return the SID as a numbered list of interactions using the format:
  Subject A -> Subject B: Message (noun form)

### 2. Subject Behavior Diagram (SBD):
- For each subject, describe their **internal behavior** as a sequence of states.
- Use the following state types and label each step explicitly:
  - **StartState**: the beginning of the behavior
  - **ReceiveState**: receiving a message
  - **SendState**: sending a message
  - **DoState**: internal processing or logic
  - **EndState**: conclusion of the subject's behavior
- Ensure each interaction in the SID maps to a SendState and ReceiveState in the corresponding SBDs.
- For every **ReceiveState**, also include the `From:` (sender subject) and `Msg:` (the received noun message) on the next line below it.
  - The `Msg:` must be a **noun or noun phrase only**, matching the noun form used in the SID (e.g., "Order", "Appointment Confirmation").  
  - Do **not** use verbs, verb phrases, or action words (e.g., "send payment", "check availability").

### Additional Constraint:
- **Ensure all outbound communications (e.g., confirmation messages or emails) are explicitly modeled as messages received by their final recipient.**
  - For example, if the Email System generates a "Confirmation Email", then the Customer must also receive it.
  - This should be reflected clearly in both the SID and the SBDs.
  
###Additional Modeling Requirements:
For every **SendState**, include the following on the next lines:

-To: the subject to whom the message is sent

-Msg: the noun-form message being sent (must match the SID)

For every **DoState**, include a Description: field that outlines the internal logic or condition that leads to the next state. This should capture possible outcomes necessary to exit the DoState.

For every **StartState** that is not a **ReceiveState** or **SendState**, provide a Description: that briefly explains the reason or trigger for entering this state (e.g., a decision or internal motivation).

### Use the following format (example based on a healthcare scenario):

### Example Scenario:
A patient books an appointment using a healthcare app.  
The app sends the appointment request to the hospital system.  
The hospital system checks the doctor’s availability.  
If the doctor is available, it confirms the appointment.  
The app then sends a confirmation message to the patient.

### Subjects:
- Patient
- Healthcare App
- Hospital System

### SID:
1. Patient -> Healthcare App: Appointment Request  
2. Healthcare App -> Hospital System: Appointment Request  
3. Hospital System -> Healthcare App: Appointment Confirmation  
4. Healthcare App -> Patient: Appointment Confirmation

### SBD:

#### Patient:
1. StartState:  Decide to make appointment
   Description: initiate appointment
2. SendState: Send Appointment Request to Healthcare App
   To:  Healthcare App
   Msg: Appointment Request
3. ReceiveState: Receive Appointment Confirmation from Healthcare App  
   From: Healthcare App  
   Msg: Appointment Confirmation  
4. EndState: Appointment booked

#### Healthcare App:
1. StartState: Receive Appointment Request from Patient  
   From: Patient  
   Msg: Appointment Request  
2. SendState: Send Appointment Request to Hospital System
   To:  Hospital System
   Msg: Appointment Request
3. ReceiveState: Receive Appointment Confirmation from Hospital System  
   From: Hospital System  
   Msg: Appointment Confirmation  
4. SendState: Send Appointment Confirmation to Patient 
   To:  Patient
   Msg: Appointment Confirmation
5. EndState: Confirmation sent

#### Hospital System:
1. StartState: Receive Appointment Request via Healthcare App  
   From: Healthcare App  
   Msg: Appointment Request  
2. DoState: Check doctor availability
   Description: check availability
3. SendState: Send Appointment Confirmation to Healthcare App
   To:  Healthcare App
   Msg: Appointment Confirmation
4. EndState: Done

Now, do the same for the scenario provided above.
"""

# Make the API call
completion = client.chat.completions.create(
    messages=[{"role": "user", "content": prompt}],
    model=model,
    temperature=0.5
)

# Output
print(completion.choices[0].message.content)

### Scenario:
A customer places an order on an e-commerce website.
The system checks inventory.
If the item is in stock, customer is informed to pay for the product.
Customer pays for the order.
The payment gateway confirms the payment transaction.
Finally, the system sends order confirmation email to the customer.

### Subjects:
- Customer
- E-commerce Website
- Inventory System
- Payment Gateway
- Email System

### SID:
1. Customer -> E-commerce Website: Order
2. E-commerce Website -> Inventory System: Inventory Check Request
3. Inventory System -> E-commerce Website: Inventory Check Response
4. E-commerce Website -> Customer: Payment Request
5. Customer -> Payment Gateway: Payment
6. Payment Gateway -> E-commerce Website: Payment Confirmation
7. E-commerce Website -> Email System: Order Confirmation Email
8. Email System -> Customer: Order Confirmation Email

### SBD:

#### Customer:
1. StartState: Decide to place order
   Description: initiate order
2. SendState: Send Order to E-co

In [1]:
import re

# saved response from LLM in variable below named mistral_output, 
# otherwise comment it out and do mistral_output = completion.choices[0].message.content

mistral_output ="""
### Scenario:
A customer places an order on an e-commerce website.
The system checks inventory.
If the item is in stock, customer is informed to pay for the product.
Customer pays for the order.
The payment gateway confirms the payment transaction.
Finally, the system sends order confirmation email to the customer.

### Subjects:
- Customer
- E-commerce Website
- Inventory System
- Payment Gateway
- Email System

### SID:
1. Customer -> E-commerce Website: Order
2. E-commerce Website -> Inventory System: Inventory Check Request
3. Inventory System -> E-commerce Website: Inventory Check Response
4. E-commerce Website -> Customer: Payment Request
5. Customer -> Payment Gateway: Payment
6. Payment Gateway -> E-commerce Website: Payment Confirmation
7. E-commerce Website -> Email System: Order Confirmation Email
8. Email System -> Customer: Order Confirmation Email

### SBD:

#### Customer:
1. StartState: Decide to place order
   Description: initiate order
2. SendState: Send Order to E-commerce Website
   To:  E-commerce Website
   Msg: Order
3. ReceiveState: Receive Payment Request from E-commerce Website
   From: E-commerce Website
   Msg: Payment Request
4. SendState: Send Payment to Payment Gateway
   To:  Payment Gateway
   Msg: Payment
5. ReceiveState: Receive Order Confirmation Email from Email System
   From: Email System
   Msg: Order Confirmation Email
6. EndState: Order confirmed

#### E-commerce Website:
1. StartState: Receive Order from Customer
   From: Customer
   Msg: Order
2. SendState: Send Inventory Check Request to Inventory System
   To:  Inventory System
   Msg: Inventory Check Request
3. ReceiveState: Receive Inventory Check Response from Inventory System
   From: Inventory System
   Msg: Inventory Check Response
4. DoState: Check inventory status
   Description: evaluate inventory status
5. SendState: Send Payment Request to Customer
   To:  Customer
   Msg: Payment Request
6. ReceiveState: Receive Payment Confirmation from Payment Gateway
   From: Payment Gateway
   Msg: Payment Confirmation
7. SendState: Send Order Confirmation Email to Email System
   To:  Email System
   Msg: Order Confirmation Email
8. EndState: Order processed

#### Inventory System:
1. StartState: Receive Inventory Check Request via E-commerce Website
   From: E-commerce Website
   Msg: Inventory Check Request
2. DoState: Check inventory
   Description: determine item availability
3. SendState: Send Inventory Check Response to E-commerce Website
   To:  E-commerce Website
   Msg: Inventory Check Response
4. EndState: Inventory checked

#### Payment Gateway:
1. StartState: Receive Payment from Customer
   From: Customer
   Msg: Payment
2. DoState: Process payment
   Description: verify payment details
3. SendState: Send Payment Confirmation to E-commerce Website
   To:  E-commerce Website
   Msg: Payment Confirmation
4. EndState: Payment processed

#### Email System:
1. StartState: Receive Order Confirmation Email from E-commerce Website
   From: E-commerce Website
   Msg: Order Confirmation Email
2. SendState: Send Order Confirmation Email to Customer
   To:  Customer
   Msg: Order Confirmation Email
3. EndState: Email sent
"""
def parse_sid(sid_text):
    pattern = re.compile(r"\d+\.\s*(.+?)\s*->\s*(.+?):\s*(.+)")
    sid = []
    for line in sid_text.splitlines():
        print(f"Parsing line: {line}")
        
        match = pattern.search(line)
        if match:
            sender = match.group(1).strip()
            receiver = match.group(2).strip()
            message = match.group(3).strip()
            sid.append((f"{sender} -> {receiver}", message))
        else:
            print("No match found.")
    return sid

def extract_sbd_section(full_text):
    lines = full_text.splitlines()
    sbd_start = None
    for i, line in enumerate(lines):
        if "### SBD" in line:
            sbd_start = i + 1
            break
    if sbd_start is None:
        print("No SBD section found!")
        return ""

    sbd_lines = []
    for line in lines[sbd_start:]:
        if line.strip().startswith("### Explanation"):
            break
        sbd_lines.append(line)

    return "\n".join(sbd_lines).strip()



def parse_sbd(sbd_text):
    print("=== SBD Text to parse ===")
    print(sbd_text)
    print("=========================")
    lines = sbd_text.splitlines()

    sbd = {}
    current_actor = None
    states = {}
    current_state = None
    current_state_num = None

    actor_header_re = re.compile(r"^\s*####\s*(.+):$")
    state_header_re = re.compile(r"(\d+)\.\s+(\w+State):\s*(.*)")

    for idx, line in enumerate(lines):
        line = line.rstrip()
        if not line:
            continue

        # Detect actor header like: #### Customer:
        actor_match = actor_header_re.match(line)
        if actor_match:
            if current_actor:
                sbd[current_actor] = list(states.values())
            current_actor = actor_match.group(1).strip()
            states = {}
            current_state = None
            current_state_num = None
            continue

        if current_actor is None:
            continue  # Skip anything before the first actor header

        # Detect numbered state line like: 1. StartState: Foo
        state_match = state_header_re.match(line)
        if state_match:
            current_state_num = int(state_match.group(1))
            state_type = state_match.group(2)
            description = state_match.group(3).strip()
            current_state = {
                "num": current_state_num,
                "type": state_type,
                "description": description
            }
            states[current_state_num] = current_state
            continue

        if current_state is None:
            continue

        line_stripped = line.strip()

        # Capture From/To/Msg in SendState, ReceiveState, and StartState
        if current_state["type"] in ["SendState", "StartState"]:
            if line_stripped.startswith("To:"):
                current_state["To"] = line.split(":", 1)[1].strip()
            elif line_stripped.startswith("Msg:"):
                current_state["Msg"] = line.split(":", 1)[1].strip()

        if current_state["type"] in ["ReceiveState", "StartState"]:
            if line_stripped.startswith("From:"):
                current_state["From"] = line.split(":", 1)[1].strip()
            elif line_stripped.startswith("Msg:"):
                current_state["Msg"] = line.split(":", 1)[1].strip()

        # Capture Description or Action for DoState and StartState
        if current_state["type"] in ["DoState", "StartState"]:
            if line_stripped.startswith("Description:"):
                current_state["Description"] = line.split(":", 1)[1].strip()

    if current_actor:
        sbd[current_actor] = list(states.values())

    return sbd


# 1. Extract sections
sbd_text = extract_sbd_section(mistral_output)

# 2. Parse
parsed_sbd = parse_sbd(sbd_text)

sid_section_start = mistral_output.find("### SID:")
sid_section_end = mistral_output.find("### SBD:")
sid_text = mistral_output[sid_section_start:sid_section_end].replace("### SID:", "").strip()
parsed_sid = parse_sid(sid_text)

# 3. Assemble full process
process = {
    "SID": parsed_sid,
    "SBD": parsed_sbd
}

print("Parsed SBD::", parsed_sbd)
print("Parsed SID::", parsed_sid) 

=== SBD Text to parse ===
#### Customer:
1. StartState: Decide to place order
   Description: initiate order
2. SendState: Send Order to E-commerce Website
   To:  E-commerce Website
   Msg: Order
3. ReceiveState: Receive Payment Request from E-commerce Website
   From: E-commerce Website
   Msg: Payment Request
4. SendState: Send Payment to Payment Gateway
   To:  Payment Gateway
   Msg: Payment
5. ReceiveState: Receive Order Confirmation Email from Email System
   From: Email System
   Msg: Order Confirmation Email
6. EndState: Order confirmed

#### E-commerce Website:
1. StartState: Receive Order from Customer
   From: Customer
   Msg: Order
2. SendState: Send Inventory Check Request to Inventory System
   To:  Inventory System
   Msg: Inventory Check Request
3. ReceiveState: Receive Inventory Check Response from Inventory System
   From: Inventory System
   Msg: Inventory Check Response
4. DoState: Check inventory status
   Description: evaluate inventory status
5. SendState: Send 

In [25]:
###########################SBD Graph##########################

import os  
os.environ["PATH"] += os.pathsep + "/home/s/smanan/.conda/envs/myenv/bin"

from graphviz import Digraph

state_styles = {
    'StartState': {"shape": "rectangle", "color": "yellow"},
    'EndState':   {"shape": "rectangle", "color": "yellow"},
    'SendState':  {"shape": "rectangle", "color": "green"},
    'ReceiveState': {"shape": "rectangle", "color": "pink"},
    'DoState':    {"shape": "rectangle", "color": "yellow"},
    'Unknown':    {"shape": "rectangle", "color": "yellow"},
}

for subject, steps in parsed_sbd.items():
    dot = Digraph(name=subject, format='pdf')
    dot.attr(rankdir='LR')

    step_map = {step["num"]: step for step in steps}
    step_nums = sorted(step_map.keys())

    # Draw nodes
    for step in steps:
        label = step["description"]

        # Determine if StartState is acting as ReceiveState
        if step["type"] == "StartState" and "From" in step and "Msg" in step and "receive" in step["description"].lower():
            style = state_styles["ReceiveState"]
        else:
            style = state_styles.get(step["type"], state_styles["Unknown"])

        dot.node(str(step["num"]), label=label, shape=style["shape"], style="filled", fillcolor=style["color"])

    # Draw transitions
    for i, step in enumerate(steps):
        current_num = str(step["num"])

        if step["type"] == "EndState":
            continue

        label = ""
        next_step_num = step_nums[i + 1] if i + 1 < len(step_nums) else None

        if step["type"] == "SendState":
            label = f"To={step.get('To', '')}\\nMsg={step.get('Msg', '')}"
        elif step["type"] == "ReceiveState":
            label = f"From={step.get('From', '')}\\nMsg={step.get('Msg', '')}"
        elif step["type"] == "DoState":
            label = step.get("Description", step["Description"])
        elif step["type"] == "StartState":
            # Support From/To/Msg in StartState if present
            from_part = f"From={step.get('From', '')}" if "From" in step else ""
            to_part = f"To={step.get('To', '')}" if "To" in step else ""
            msg_part = f"Msg={step.get('Msg', '')}" if "Msg" in step else ""

            parts = [from_part, to_part, msg_part]
            parts = [p for p in parts if p]
            
            label = "\\n".join(parts) if parts else step.get("Description", step.get("Action", step["description"]))

        if next_step_num:
            dot.edge(current_num, str(next_step_num), label=label)

    safe_subject = subject.replace(" ", "_")
    filename = f"/home/s/smanan/sbd_{safe_subject}"
    dot.render(filename, cleanup=True)


In [ ]:
import os  ###########################SID Graph##########################
os.environ["PATH"] += os.pathsep + "/home/s/smanan/.conda/envs/myenv/bin"

from graphviz import Digraph

def draw_sid_graph(sid_list, output_path="sid_graph"):
    dot = Digraph(comment="SID - Sequence Interaction Diagram")
    dot.attr(rankdir='LR',splines='polyline')  # Left to right direction

    for interaction, action in sid_list:
        source, target = [s.strip() for s in interaction.split("->")]
        dot.node(source, source, shape="box", style="filled", fillcolor="lightblue")
        dot.node(target, target, shape="box", style="filled", fillcolor="lightgreen")
        dot.edge(source, target, label=action)

    # Render graph to file
    dot.render(output_path, format='png', cleanup=True)
    print(f"SID graph saved to: {output_path}.png")

In [ ]:
draw_sid_graph(process["SID"], output_path="/home/s/smanan/sid_graph")